Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [14]:
NAME = "Lyubomira Dimitrova"
COLLABORATORS = "Maryna Charniuk, Dung Nguyen"

---

# Inverses Hashing

Hashtabellen werden ineffizient, wenn die verwendete Hashfunktion viele Kollisionen verursacht. Ein Hacker, der die Hashfunktion kennt, kann diese Tatsache ausnutzen, um absichtlich ineffiziente Anfragen zu konstruieren: Er wählt die in der Anfrage verwendeten Schlüssel gezielt so, dass alle auf den gleichen Hashwert abgebildet werden ("adversarial keys"). Werden viele ineffiziente Anfragen dieser Art gleichzeitig gestartet, kann die angegriffene Webseite (die die gegebene Hashfunktion intern benutzt) zusammenbrechen ("denial of service"). In der Praxis verhindert man dies durch die Verwendung von universellem Hashing, indem man die Hashfunktion per Zufall aus einem großen Pool erlaubter Hashfunktionen auswählt. Dann ist es nicht mehr möglich, im Vorhinein ein ungünstiges Set von Schlüsseln zu konstruieren. Für diese Übungsaufgabe wollen wir aber annehmen, dass stets die folgende einfache Hashfunktion verwendet wird:
```python
def hhash(s): # s ist ein Schlüssel vom Typ string
    h = 0     # der Hashwert wird mit 0 initialisiert
    for k in s:
        h = 23*h + ord(k)  # Aktualisieren des Hashs mit dem Zeichencode
    return h
```
Dabei gibt `ord(k)` den Zeichencode des Zeichens k zurück. Schreiben Sie eine Funktion um mindestens 16 Schlüssel (Strings) der Länge 4 zu finden, die alle den gleichen Hashwert haben. Geben Sie diese Schlüssel zurück und beschreiben Sie (Python docstring im Funktionskopf), wie Sie vorgegangen sind, um diese Schlüssel zu finden. Hinweis: Beginnen Sie damit, Kollisionen mit Schlüsseln der Länge 2 zu konstruieren und verwenden Sie diese Ergebnisse zur Konstruktion von Schlüsseln der Länge 4.

In [8]:
def hhash(s):
    h = 0
    for k in s:
        h = 23*h + ord(k)
    return h


def findAdversialKeys():
    """
    Finds >=16 keys of length 4, which have the same hash value when using hhash(s).
    
    Returns:
    keys - list - a list of keys which have the same hash value
    
    Approach:
    1. Create an alphabet, in the ASCII-range (48, 123), which is arbitrary, 
    but should contain at least 70 characters.
    2. Create a list (pairs) of all possible two-letter combinations of the
    alphabet.
    3. Create an empty table, where collisions will be collected as values to 
    a particular hash value.
    4. Fill the table: 
        - Go through all pairs
        - For each pair calculate the hash value
        - Check if the value is already a key in the table. If yes, add the key 
        to the collisions list. If not, create a new table entry, with the hash
        value as key, and as value, start a collisions list (containing only
        one element).
    5. Find a collisions list that contains four or more keys.
    6. Create a list (keys) of all possible combinations of these two-letter keys.
    These new four-letter keys will also have the same hash value, since the hhash
    function can be reduced to (23^3 * a) + (23^2 * b) + (23 * c) + d for any four-letter 
    string abcd.
    7. After one such collisions list is found, break and return.
    
    """
    alphabet = [chr(ascii) for ascii in range(48, 123)]   # 0-z
    
    pairs = [first + second for first in alphabet for second in alphabet]
    
    table = {}
    for p in pairs:
        h = hhash(p)
        if table.get(h, None):
            table[h].append(p)
        else:
            table.setdefault(h, [p])
    
    collisions = table.values()
    keys = []
    for x in collisions:
        if len(x) >= 4:
            keys = [first + second for first in x for second in x]
            break   # theoretically collecting collisions of >= 16 strings is also possible
    
    return keys

print(findAdversialKeys())

['0u0u', '0u1^', '0u2G', '0u30', '1^0u', '1^1^', '1^2G', '1^30', '2G0u', '2G1^', '2G2G', '2G30', '300u', '301^', '302G', '3030']


In [9]:
print("Testing inverse hashing.")
adv_keys = findAdversialKeys()
print("done")

Testing inverse hashing.
done


# 2. Hash-Table

Definieren Sie eine Klasse `HashTable`, die eine Hash-Tabelle der Größe `size` als geschachtelte Liste (Liste in einer Liste) implementiert (Bsp. `[[], [], [], [], [], [], [], [], [], []]`). Implementieren Sie die folgenden drei Methoden:

  - `set(key, value)`: Um ein neues Schlüssel-Wert-Paar zu der Tabelle hinzuzufügen
  - `get(key)`: Um den Wert zu einem Schlüssel zurück zu geben. Raise a KeyError when the given key isn't in the Hash Table.
  - `remove(key)`: Um ein Schlüssel-Wert-Paar aus der Tabelle zu entfernen. Befindet sich der Schlüssel nicht in der Liste soll eine `KeyError` Ausnahme ausgelöst werden.
 

In [10]:
# function from the previous assignment
def my_hash(item, s):
    if isinstance(item, int):
        return item % s
    if isinstance(item, str):
        sum_chars = 0
        for char in item:
            sum_chars += ord(char)
        return sum_chars % s
    

class KeyValuePair(object):

    def __init__(self, key, value):
        self.key = key
        self.value = value

        
class HashTable(object):

    def __init__(self, size):
        self.size = size
        self.table = [[] for i in range(self.size)]
        self.keys = set()
        
    def set_(self, key, value):
        bucket = my_hash(key, self.size)
        
        if key in self.keys:
            for pair in self.table[bucket]:
                if pair.key == key:
                    pair.value = value
                    break
        else:
            self.table[bucket].append(KeyValuePair(key, value))
            self.keys.add(key)

    
    def get(self, key):
        bucket = my_hash(key, self.size)
        
        if not self.table[bucket]:
            raise KeyError ('key is not in Hash Table')
    
        for pair in self.table[bucket]:
            if pair.key == key:
                return pair.value
    
    
    def remove(self, key):
        if key not in self.keys:
            raise KeyError ('key is not in Hash Table')
        
        bucket = my_hash(key, self.size)
        
        for pair in self.table[bucket]:
            if pair.key == key:
                self.table[bucket].remove(pair)
        

In [11]:
hash_table = HashTable(10)


Schreiben Sie Einheitentests um die Funktionen `insert`, `get` und `remove` zu testen. Überprüfen Sie folgendes Verhalten:

 1. `get` auf eine leere Tabelle
 2. `set` auf eine leere Tabelle
 3. `set` auf eine nicht-leere Tabelle mit nicht vorhandenem Schlüssel
 4. `set` mit bereits vorhandenem Schlüssel (Der Wert sollte aktualisiert werden)
 5. `remove` auf einen vorhandenen Schlüssel
 6. `remove`auf einen nicht vorhandenen Schlüssel

Geben Sie **Success: test_№** wenn der Test erfolgreich durchläuft (Bsp. **Success: test_1** für den ersten Test).

In [12]:
import unittest

class HashTest(unittest.TestCase):
    
    def test_1_get_on_empty_hash_table(self):
        hash_table = HashTable(10)
        with self.assertRaises(KeyError):
            hash_table.get(3)
        print('Success: test_1')
        
    def test_2_set_on_empty_hash_table(self):
        hash_table = HashTable(10)
        hash_table.set_(2,20)
        bucket = my_hash(2, 10)
        self.assertTrue(hash_table.table[bucket] is not None)    # the corresponding bucket should not be empty
        print('Success: test_2')
        
    def test_3_set_on_non_empty_hash_table(self):
        hash_table = HashTable(10)
        hash_table.set_(2,20)
        hash_table.set_(4,40)
        self.assertTrue(hash_table.table[my_hash(4, 10)] is not None)   # the corresponding bucket should not be empty
        print('Success: test_3')
        
    def test_4_set_on_a_key_that_already_exists(self):
        hash_table = HashTable(10)
        bucket = my_hash(2, 10)
        hash_table.set_(2,20)
        old_bucket = hash_table.table[bucket]
        hash_table.set_(2,40)
        self.assertEqual(len(hash_table.table[bucket]), len(old_bucket))  # the size of the bucket should not change
        print('Success: test_4')
        
    def test_5_remove_on_a_key_that_already_exists(self):
        hash_table = HashTable(10)
        hash_table.set_(2, 20)
        bucket = my_hash(2, 10)
        old_bucket = hash_table.table[bucket][:]
        hash_table.remove(2)
        self.assertEqual(len(hash_table.table[bucket]), len(old_bucket) - 1)  # the size of the bucket should change
        print('Success: test_5')
        
    def test_6_remove_on_a_key_that_does_not_exist(self):
        hash_table = HashTable(10)
        with self.assertRaises(KeyError):
            hash_table.remove(3)
        print('Success: test_6')
            
unittest.main(argv=['first-arg-is-ignored'], exit=False)

......

Success: test_1
Success: test_2
Success: test_3
Success: test_4
Success: test_5
Success: test_6



----------------------------------------------------------------------
Ran 6 tests in 0.007s

OK


In [13]:
test_suite = unittest.TestLoader().loadTestsFromTestCase(HashTest)
import os
test_result = unittest.TextTestRunner(stream = open(os.devnull, 'w')).run(test_suite)


Success: test_1
Success: test_2
Success: test_3
Success: test_4
Success: test_5
Success: test_6
